# Getting data from Twitter

In [6]:
from datetime import datetime
import time

import pandas as pd
import twint
import nest_asyncio
nest_asyncio.apply()


users_downloaded_at = '2021-11-18'
download_tweets_from = '2021-08-12'
download_tweets_to = '2021-08-19'

pd.options.display.max_columns = 500

## Users I want

In [3]:
handles = [
    'BarackObama', 'MichelleObama', 'Oprah', 'realmadrid', 'fcbarcelona', 'sanchezcastejon', 'theonion'
]

## Getting Users

In [4]:
results = []
unfound_accounts = []
not_worked = []
for i, username in enumerate(handles):
    if i % 100 == 0:
        print(f'Done with {i}')
    
    c = twint.Config()
    c.Username = username
    c.Pandas = True
    c.Hide_output = True
    
    try:
        twint.run.Lookup(c)
    
        results.append(twint.storage.panda.User_df)
    except ValueError as e:
        if 'Cannot find twitter account' in str(e):
            unfound_accounts.append(username)
        else:
            raise
    except twint.token.RefreshTokenException:
        not_worked.append(username)
        time.sleep(60)
        
        try:
            twint.run.Lookup(c)
            results.append(twint.storage.panda.User_df)
        except:
            print('Getting refresh errors!')

results = pd.concat(results)
results.to_csv(f'{datetime.today().strftime("%Y-%m-%d")}-handles-data.csv', index=False)

print(unfound_accounts)
print(not_worked)

Done with 0
[]
[]


## Getting tweets

In [9]:
user_details = pd.read_csv(f'{users_downloaded_at}-handles-data.csv').drop_duplicates('id')
user_details

,id,name,username,bio,url,join_datetime,join_date,join_time,tweets,location,following,followers,likes,media,private,verified,avatar,background_image
0,813286,Barack Obama,BarackObama,"Dad, husband, President, citizen.",https://t.co/FPuJ0CENpo,2007-03-05 22:08:25 UTC,2007-03-05,22:08:25 UTC,16310,"Washington, DC",588155,130169227,0,2346,False,True,https://pbs.twimg.com/profile_images/132964752...,https://pbs.twimg.com/profile_banners/813286/1...
1,409486555,Michelle Obama,MichelleObama,Girl from the South Side and former First Lady...,https://t.co/HPYP9ad13Y,2011-11-10 20:13:01 UTC,2011-11-10,20:13:01 UTC,1813,"Washington, DC",16,21076891,186,476,False,True,https://pbs.twimg.com/profile_images/136674780...,https://pbs.twimg.com/profile_banners/40948655...
2,19397785,Oprah Winfrey,Oprah,NaN,https://t.co/IKOnfAha8E,2009-01-23 15:18:34 UTC,2009-01-23,15:18:34 UTC,13478,NaN,325,43163768,243,970,False,True,https://pbs.twimg.com/profile_images/112335936...,https://pbs.twimg.com/profile_banners/19397785...
3,14872237,Real Madrid C.F.,realmadrid,⚽️ Cuenta oficial del Real Madrid C.F. 🏆@realm...,https://t.co/s1ejWbkVKF,2008-05-22 19:25:51 UTC,2008-05-22,19:25:51 UTC,78050,"Madrid, Spain",65,39141838,229,30012,False,True,https://pbs.twimg.com/profile_images/128419774...,https://pbs.twimg.com/profile_banners/14872237...
4,96951800,FC Barcelona,FCBarcelona,#ForçaBarça! Join us: @fcbarcelona_cat @fcbarc...,https://t.co/vboAVBT1by,2009-12-15 09:53:50 UTC,2009-12-15,09:53:50 UTC,119567,CULERS MEMBERSHIP! 👉,103,38798702,3626,74593,False,True,https://pbs.twimg.com/profile_images/140974084...,https://pbs.twimg.com/profile_banners/96951800...
5,68740712,Pedro Sánchez,sanchezcastejon,Presidente del Gobierno de #España. Padre. Sec...,https://t.co/h8b36qazCM,2009-08-25 16:45:56 UTC,2009-08-25,16:45:56 UTC,30497,NaN,6039,1602235,4813,7885,False,True,https://pbs.twimg.com/profile_images/141011963...,https://pbs.twimg.com/profile_banners/68740712...
6,14075928,The Onion,TheOnion,America's Finest News Source.,https://t.co/EIF811CxOS,2008-03-04 02:48:37 UTC,2008-03-04,02:48:37 UTC,84757,NaN,8,11697662,3,45724,False,True,https://pbs.twimg.com/profile_images/875392068...,https://pbs.twimg.com/profile_banners/14075928...


In [10]:
results = []
not_worked = []
unfound_accounts = []
for i, username in enumerate(user_details.username.dropna().unique()):
    if i % 100 == 0:
        print(f'Done with {i}')
    
    c = twint.Config()
    c.Username = username
    c.Since = download_tweets_from
    c.Until = download_tweets_to
    c.Pandas = True
    c.Hide_output = True
    
    try:
        twint.run.Profile(c)

        results.append(twint.storage.panda.Tweets_df)
    except ValueError as e:
        if 'Cannot find twitter account' in str(e):
            unfound_accounts.append(username)
        else:
            raise
    except twint.token.RefreshTokenException:
        not_worked.append(username)
        time.sleep(60)

        try:
            twint.run.Profile(c)
            results.append(twint.storage.panda.Tweets_df)
        except:
            print('Getting refresh errors!')
        
    
results = pd.concat(results)
results.to_csv(f'{users_downloaded_at}-{download_tweets_from}-{download_tweets_to}-tweets-data.csv', index=False)

Done with 0
[!] No more data! Scraping will stop now.
found 1 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 3 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 12 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
